# This script analysis the EMG data collect using two EMG channels and 6 IMU signals.

## The data is pre-processed: first 20-500 Hz band-pass filter was applied on the raw data (1000 Hz); second, a window of 60ms with stride of 20 ms was used (so sampling rate is 50 Hz);
## In each window, a new value was computed as [ 1/n * (x1^2+x2^2+...xT^2)^(1/2) ]

# label: 1-->grasp; 2-->release;
# emg1: shen ji qun; emg2: qu ji qun;

In [1]:
from pre_all import top_data_dir
import  numpy as np

2024-07-08 14:11:41 - Start Program

pre_all: Running from CMD.
common_dl.py: Using CUDA.


In [2]:
import matplotlib.pyplot as plt

In [3]:
filename=top_data_dir+'HaiMeiKang/ddd.npz'

In [4]:
data=np.load(filename, allow_pickle=False)

In [5]:
emg1=data['emg1']
emg2=data['emg2']
label=data['keyboard_action'] # label using keyboard
label2=data['emg_action'] # label using emg data
hand_shape=data['hand_position'] # shape of hand

In [76]:
%matplotlib notebook
plt.figure()
plt.plot(emg1,label='emg1')
plt.plot(emg2,label='emg2')
plt.plot((label+1)*100,label='label')
plt.show()

<IPython.core.display.Javascript object>

In [51]:
label.shape[0]

19203

In [7]:
%matplotlib notebook
fig,ax=plt.subplots(1,1)
ax.plot(label,label='label')

<IPython.core.display.Javascript object>

In [8]:
# extract the beginning and ending of event 1 and event 2;
starts1=[]
ends1=[]
starts2=[]
ends2=[]
for i in range(1,label.shape[0]-2):
    if label[i]==1:
        if label[i-1]!=1:
            starts1.append(i)
        if label[i+1]!=1:
            ends1.append(i)
    
    if label[i]==2:
        if label[i-1]!=2:
            starts2.append(i)
        if label[i+1]!=2:
            ends2.append(i)
            

In [14]:
# Remove labels that last less than 50 which corresponding to situation that exoskeleton no response when pressing 1)
# exoskeleton works fine when pressing 2;
starts11=[]
ends11=[]
for i in range(len(starts1)):
    if ends1[i]-starts1[i]>50:
        starts11.append(starts1[i])
        ends11.append(ends1[i])

In [12]:
ends1

[614,
 1025,
 1091,
 1243,
 1689,
 2010,
 2077,
 2283,
 2750,
 3139,
 3204,
 3351,
 3778,
 3828,
 3967,
 4398,
 4435,
 4601,
 4704,
 5241,
 5852,
 6236,
 6285,
 6488,
 6806,
 6869,
 7015,
 7410,
 7641,
 8014,
 8053,
 8219,
 8573,
 8632,
 8799,
 9301,
 9341,
 9476,
 9967,
 10137,
 10668,
 10742,
 11107,
 11158,
 11321,
 11641,
 11680,
 11839,
 12503,
 12721,
 13069,
 13228,
 13963,
 14142,
 14482,
 14652,
 14988,
 16442,
 16641,
 17056,
 17562,
 17981,
 18265,
 18380,
 18592]

In [13]:
starts1

[453,
 985,
 1054,
 1104,
 1523,
 1972,
 2037,
 2104,
 2613,
 3101,
 3166,
 3219,
 3743,
 3802,
 3852,
 4359,
 4412,
 4448,
 4679,
 5098,
 5721,
 6187,
 6250,
 6325,
 6755,
 6832,
 6884,
 7372,
 7424,
 7979,
 8018,
 8056,
 8524,
 8586,
 8643,
 9252,
 9305,
 9357,
 9877,
 9995,
 10391,
 10703,
 11060,
 11121,
 11173,
 11605,
 11644,
 11695,
 12392,
 12554,
 12991,
 13084,
 13870,
 13978,
 14416,
 14496,
 14976,
 16403,
 16468,
 16903,
 17349,
 17824,
 18227,
 18355,
 18433]

In [68]:
%matplotlib notebook
fig,ax=plt.subplots(1,1)
ax.plot(label,label='label')

for i in range(len(starts11)):
    ax.axvline(x=starts11[i], color='green', linestyle='--')
    ax.axvline(x=ends11[i], color='yellow', linestyle='--')
    
for i in range(len(starts2)):
    ax.axvline(x=starts2[i], color='red', linestyle='--')
    ax.axvline(x=ends2[i], color='black', linestyle='--')
plt.show()

<IPython.core.display.Javascript object>

In [58]:
# extract data belonging to grasping (1) and release (2)
grasps=[]
releases=[]
for i in range(len(starts11)):
    tmp1=emg1[starts11[i]:ends11[i]]
    tmp2=emg2[starts11[i]:ends11[i]]
    grasps.append(np.asarray([tmp1,tmp2]))

for i in range(len(starts2)):
    tmp1=emg1[starts2[i]:ends2[i]]
    tmp2=emg2[starts2[i]:ends2[i]]
    releases.append(np.asarray([tmp1,tmp2]))

In [75]:
# visualize emg1 and emg2 during grasping
%matplotlib notebook
fig,ax=plt.subplots(5,5)
for i in range(5):
    for j in range(5):  
        ax[i,j].plot(grasps[i*5+j][0,:])
        ax[i,j].plot(grasps[i*5+j][1,:])
plt.legend(['emg1','emg2'])
plt.show()

<IPython.core.display.Javascript object>

In [67]:
# visualize emg1 and emg2 during releasing
%matplotlib notebook
fig,ax=plt.subplots(5,5)
for i in range(5):
    for j in range(5):  
        ax[i,j].plot(releases[i*5+j][0,:])
        ax[i,j].plot(releases[i*5+j][1,:])
plt.show()

<IPython.core.display.Javascript object>

In [74]:
# visualize (emg2-emg1) for grasp and release
%matplotlib notebook
fig,ax=plt.subplots(5,5)
for i in range(5):
    for j in range(5):  
        ax[i,j].plot(releases[i*5+j][1,:]-releases[i*5+j][0,:])
        ax[i,j].plot(grasps[i*5+j][1,:]-grasps[i*5+j][0,:])
plt.legend(['release','grasp'])
plt.show()

<IPython.core.display.Javascript object>

In [72]:
releases[0][0,:]-releases[0][1,:]

array([-250, -250, -250, -250, -250, -250, -250, -250, -250, -250, -250,
       -250, -250, -250, -250, -250, -250, -250, -250, -250, -250, -250,
       -250, -250, -250, -122, -122, -122, -122, -122, -122, -122, -122,
       -122, -122, -122, -122, -122, -122, -122, -122, -156, -156, -156,
       -156, -156, -156, -156, -156, -156, -156, -156, -156, -156, -156,
       -156, -213, -213, -213, -213, -213, -213, -213, -213, -213, -213,
       -213, -213, -213, -218, -211, -211, -211, -211, -211, -211, -211,
       -211, -211, -211, -211, -211, -211, -131, -131, -131, -131, -131,
       -131, -131, -131, -131, -131, -131, -131, -131, -161, -161, -161,
       -161, -161, -161, -161, -161, -161, -161, -161, -161, -161, -230,
       -230, -230, -230, -230, -230, -230, -230, -230, -230, -230, -230,
       -127, -127, -127, -127, -127, -127, -127, -127, -127, -127, -127,
       -127, -127, -263, -263, -263, -263, -263, -263, -263, -263, -263,
       -263, -263, -263, -263, -166, -166, -166, -1